<a href="https://colab.research.google.com/github/gabyx7677/Proyectos/blob/main/PredicionesDatosEl%C3%A9ctricos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -----------------------------
# CÓDIGO 1: ENTRENAMIENTO DEL MODELO
# -----------------------------

# Importar librerías necesarias
import pandas as pd  # Para manejo de datos
import numpy as np  # Para operaciones numéricas
import matplotlib.pyplot as plt  # Para visualización de datos
from sklearn.preprocessing import MinMaxScaler  # Para normalizar los datos
from tensorflow.keras.models import Sequential  # Para crear el modelo LSTM
from tensorflow.keras.layers import LSTM, Dense  # Capas del modelo LSTM
import joblib  # Para guardar el scaler

# Cargar archivo CSV con datos originales
df = pd.read_csv("/content/electricityConsumptionAndProductioction.csv")

# Convertir la columna "DateTime" a tipo datetime
df["DateTime"] = pd.to_datetime(df["DateTime"])

# Establecer la columna DateTime como índice
df.set_index("DateTime", inplace=True)

# Seleccionar solo la columna "Consumption" como variable a predecir
data = df[["Consumption"]].values

# Inicializar y aplicar MinMaxScaler para normalizar los datos
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)  # Normaliza entre 0 y 1

# Guardar el scaler para uso futuro con nuevos datos
joblib.dump(scaler, "scaler_consumo.save")

# Crear secuencias para entrenar el modelo (ventanas deslizantes)
def create_sequences(data, seq_length=100):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# Definir longitud de la secuencia y crear datos X, y
seq_length = 100
X, y = create_sequences(data_scaled, seq_length)

# Separar 80% para entrenamiento y 20% para prueba
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Definir arquitectura del modelo LSTM
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(seq_length, 1)),
    LSTM(50, return_sequences=False),
    Dense(25),
    Dense(1)
])

# Compilar el modelo usando el optimizador Adam y error cuadrático medio
model.compile(optimizer='adam', loss='mse')
model.summary()  # Muestra resumen del modelo

# Entrenar el modelo con datos de entrenamiento y validación
history = model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))

# Guardar el modelo entrenado
model.save("modelo_consumo_lstm.h5")

# Crear predicciones para 365 días futuros
future_days = 365
future_input = data_scaled[-seq_length:]

predictions = []
for _ in range(future_days):
    future_input = future_input.reshape(1, seq_length, 1)
    pred = model.predict(future_input)[0]
    predictions.append(pred)
    future_input = np.append(future_input[0][1:], pred).reshape(seq_length, 1)

# Desnormalizar las predicciones
predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

# Crear fechas futuras para graficar
future_dates = pd.date_range(df.index[-1], periods=future_days+1, freq="D")[1:]

# Visualizar resultados
plt.figure(figsize=(10,5))
plt.plot(df.index, df["Consumption"], label="Datos reales")
plt.plot(future_dates, predictions, label="Predicción", linestyle="dashed")
plt.legend()
plt.show()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 100, 50)             │          10,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 50)                  │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 25)                  │           1,275 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              26 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 31,901 (124.61 KB)

 Trainable params: 31,901 (124.61 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
2704/2704 ━━━━━━━━━━━━━━━━━━━━ 248s 90ms/step - loss: 0.0106 - val_loss: 0.0011
Epoch 2/20
2704/2704 ━━━━━━━━━━━━━━━━━━━━ 259s 89ms/step - loss: 6.5647e-04 - val_loss: 8.5934e-04
Epoch 3/20
2704/2704 ━━━━━━━━━━━━━━━━━━━━ 259s 87ms/step - loss: 5.3801e-04 - val_loss: 8.8684e-04
Epoch 4/20
2704/2704 ━━━━━━━━━━━━━━━━━━━━ 242s 89ms/step - loss: 4.9936e-04 - val_loss: 7.9454e-04
Epoch 5/20
2416/2704 ━━━━━━━━━━━━━━━━━━━━ 23s 80ms/step - loss: 4.5399e-04

In [ ]:
# -----------------------------
# CÓDIGO 2: USO DEL MODELO ENTRENADO CON NUEVOS DATOS
# -----------------------------

# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
import joblib  # Para cargar el scaler

# Cargar nueva base de datos
df_new = pd.read_csv(r"/content/datos_sinteticos_consumo_produccion.csv")
df_new["DateTime"] = pd.to_datetime(df_new["DateTime"])
df_new.set_index("DateTime", inplace=True)

# Seleccionar la columna de consumo
data_new = df_new[["Consumption"]].values

# Cargar el scaler previamente guardado (NO volver a entrenarlo con fit_transform)
scaler = joblib.load("scaler_consumo.save")
data_new_scaled = scaler.transform(data_new)

# Crear secuencias para el nuevo conjunto de datos
def create_sequences(data, seq_length=100):
    X = []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
    return np.array(X)

seq_length = 100
X_new = create_sequences(data_new_scaled, seq_length)

# Cargar modelo previamente entrenado
model = load_model(r"C:\\Users\\gabri\\Pandas\\modelo_consumo_lstm.h5", compile=False)

# Realizar predicciones
predictions = model.predict(X_new)
predictions = scaler.inverse_transform(predictions)  # Desnormalizar

# Obtener fechas para las predicciones
pred_dates = df_new.index[seq_length:]

# Graficar resultados
plt.figure(figsize=(12,6))
plt.plot(df_new.index, df_new["Consumption"], label="Consumo real (nuevo)")
plt.plot(pred_dates, predictions, label="Predicción", linestyle="dashed")
plt.title("Predicción de consumo con base de datos nueva")
plt.xlabel("Fecha")
plt.ylabel("Consumo")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()